# **Линейная регрессия: численное решение**

✍ Ранее мы с вами установили, что аналитическое решение линейной регрессии — метод наименьших квадратов — имеет несколько недостатков. Основной из них — большая вычислительная сложность обращения матриц.

Хотелось бы получить метод, который справляется с этим недостатком. Для этого давайте вернёмся на шаг назад и вспомним, как вообще звучит постановка задачи поиска параметров модели.

В самом простом двумерном случае мы пытаемся найти такие коэффициенты  уравнения прямой w0 и w1, чтобы средний квадрат ошибки (MSE) был минимален.

![](data\f38.png)

Гауссу удалось найти общее решение для этой оптимизационной задачи и вывести формулу метода наименьших квадратов для поиска коэффициентов.

*Но можно пойти другим путём, не аналитическим (вывод формулы «в лоб»), а численным (итерационное приближение функции к минимуму).*

Самым популярным численным методом оптимизации, используемым в машинном обучении, является **алгоритм градиентного спуска**.

**Градиентный спуск (Gradient descent)** — самый используемый *алгоритм* минимизации функции потерь. Он применяется почти в каждой модели машинного обучения и является наиболее простым в реализации из всех методов численной оптимизации.
***
Начнём обзор алгоритма немного издалека.

Представим, что мы находимся на некоторой холмистой местности и нам надо добраться до самой низкой точки, но делаем мы это вслепую, то есть не знаем сам ландшафт. Назовём эту точку целью.

![](https://lms.skillfactory.ru/assets/courseware/v1/955db3eb396297eabf11b61019fb49d5/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-ml2-4_1.png)

Давайте опишем, как мы будем искать эту точку. Для начала необходимо задать начальную точку, из которой мы, собственно, стартуем. Далее мы двигаемся в сторону крутизны склона. Если склон круче справа, надо сделать шаг вправо, если склон круче слева, надо сделать шаг влево. Повторяем шаги до тех пор, пока не достигнем самой низкой точки.

У вас мог возникнуть вопрос: как определить, что мы достигли цели — самой низкой точки? В этой точке крутизна склона с обеих сторон равна 0 или близка к нему (ровная поверхность). Можно использовать эту информацию как точку остановки нашего алгоритма.
***
Описанный нами алгоритм можно перевести на язык математики. Он то и будет называться **алгоритмом градиентного спуска**.

Наша функция потерь, которая зависит от параметров модели, — это аналогия ландшафта местности. Пространство, в котором находится ландшафт, — это пространство параметров нашей модели. То есть это система координат, в которой по осям отложены все возможные значения параметров.

В двумерном случае, когда есть только один параметр, от которого зависит функция потерь, можно построить график функции потерь. Например, для MSE, зависящей от одного параметра, график будет иметь вид **параболы**:

![](https://lms.skillfactory.ru/assets/courseware/v1/2031eae68e1cfcfbfa6642928a844ef8/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-ml2-4_2.png)

Если параметров не один, а два, то функция потерь будет графически представлена в виде **поверхности в трёхмерном пространстве**. Ниже приведён пример такой поверхности и её вид сверху в виде концентрических кругов:

![](https://lms.skillfactory.ru/assets/courseware/v1/86c47a39251c3d192a314187ec8d9043/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-ml2-4_3.png)

В общем случае, когда у нас больше параметров модели, мы будем работать в многомерном пространстве. Но в этом нет ничего страшного. Суть поиска минимума от этого не меняется, меняется только сложность функции — **структуры ландшафта**.

